In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pytorch-msssim

In [3]:
import glob
import random
import os
import numpy as np
import pandas as pd
import argparse
import math
import itertools
import sys

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

from matplotlib import pyplot as plt

from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
import sklearn

In [4]:
srgan_root = "drive/MyDrive/ImageSuperResolution/DatasetBatches/test/SRGAN_generated/"
final_root = "drive/MyDrive/ImageSuperResolution/DatasetBatches/test/final_generated_200/"
original_root = "drive/MyDrive/ImageSuperResolution/DatasetBatches/test/high/"

In [5]:
srgan_paths = sorted(glob.glob(srgan_root + "/*.*"))
final_paths = sorted(glob.glob(final_root + "/*.*"))
original_paths = sorted(glob.glob(original_root + "/*.*"))

In [6]:
len(srgan_paths),len(final_paths),len(original_paths)

(739, 739, 739)

In [7]:
eva = pd.DataFrame()

In [8]:
eva['srgan'] = srgan_paths
eva['final'] = final_paths
eva['original'] = original_paths

In [9]:
eva

,srgan,final,original
0,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
1,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
2,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
3,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
4,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
...,...,...,...
734,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
735,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
736,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...
737,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...


In [10]:
srgan = plt.imread(eva['srgan'][0])
original = plt.imread(eva['original'][0])

In [11]:
nn.functional.mse_loss

<function torch.nn.functional.mse_loss>

In [12]:
j_ = plt.imread(eva['original'][0])
k_ = plt.imread(eva['srgan'][0])
l_ = plt.imread(eva['final'][0])

In [13]:
j = Image.open(eva['original'][0])
k = Image.open(eva['srgan'][0])
l = Image.open(eva['final'][0])

In [14]:
transform = transforms.Compose(
            [
                transforms.ToTensor(),
            ]
        )

In [15]:
j = transform(j)
k= transform(k)
l= transform(l)

In [16]:
j_t = torch.from_numpy(j_.transpose((2, 0, 1))).contiguous()
k_t = torch.from_numpy(k_.transpose((2, 0, 1))).contiguous()
l_t = torch.from_numpy(l_.transpose((2, 0, 1))).contiguous()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  """Entry point for launching an IPython kernel.


In [17]:
ssim(k[np.newaxis, ...]*255., j[np.newaxis, ...]*255.), ssim(l[np.newaxis, ...]*255., j[np.newaxis, ...]*255.)

(tensor(0.4487), tensor(0.7429))

In [18]:
(np.square(k_ -j_)).mean(axis=None), (np.square(l_ -j_)).mean(axis=None)

(102.7804625, 71.29869791666667)

In [19]:
srgan_mse = []
srgan_ssim = []

final_mse = []
final_ssim = []
for i in range(len(eva)):
    j_ = plt.imread(eva['original'][i])
    k_ = plt.imread(eva['srgan'][i])
    l_ = plt.imread(eva['final'][i])
    
    ori = transform(Image.open(eva['original'][i]))[np.newaxis, ...]*255.
    sr = transform(Image.open(eva['srgan'][i]))[np.newaxis, ...]*255.
    fin = transform(Image.open(eva['final'][i]))[np.newaxis, ...]*255.
    
    srgan_mse.append((np.square(k_ - j_)).mean(axis=None))
    final_mse.append((np.square(l_ - j_)).mean(axis=None))
    
    srgan_ssim.append(ssim(sr, ori).item())
    final_ssim.append(ssim(fin, ori).item())
    
    if i%50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [20]:
len(srgan_mse), len(srgan_ssim), len(final_mse) ,len(final_ssim)

(739, 739, 739, 739)

In [21]:
eva['SRGAN_mse'] = srgan_mse
eva['FINAL_mse'] = final_mse

eva['SRGAN_ssim'] = srgan_ssim
eva['FINAL_ssim'] = final_ssim

In [22]:
eva

,srgan,final,original,SRGAN_mse,FINAL_mse,SRGAN_ssim,FINAL_ssim
0,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,102.780462,71.298698,0.448659,0.742925
1,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,98.262171,111.370181,0.373016,0.611790
2,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.561335,88.314998,0.408802,0.676006
3,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,105.399202,74.307335,0.302568,0.640555
4,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,111.062402,110.135102,0.187558,0.462170
...,...,...,...,...,...,...,...
734,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.406627,84.377056,0.400957,0.676917
735,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,94.142142,102.236035,0.195535,0.459682
736,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,108.593665,73.421621,0.391609,0.822925
737,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.530373,89.219008,0.306131,0.529648


In [23]:
def psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    return 20 * torch.log10(255.0 / torch.sqrt(mse))

In [24]:
psnr(k*255.,j*255.), psnr(l*255.,j*255.)

(tensor(12.9366), tensor(25.4249))

In [25]:
srgan_psnr = []
final_psnr = []

for i in range(len(eva)):
    ori = transform(Image.open(eva['original'][i]))[np.newaxis, ...]*255.
    sr = transform(Image.open(eva['srgan'][i]))[np.newaxis, ...]*255.
    fin = transform(Image.open(eva['final'][i]))[np.newaxis, ...]*255.
    
    srgan_psnr.append(psnr(sr, ori).item())
    final_psnr.append(psnr(fin, ori).item())
    if i%50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [26]:
eva['SRGAN_psnr'] = srgan_psnr
eva['FINAL_psnr'] = final_psnr

In [27]:
eva

,srgan,final,original,SRGAN_mse,FINAL_mse,SRGAN_ssim,FINAL_ssim,SRGAN_psnr,FINAL_psnr
0,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,102.780462,71.298698,0.448659,0.742925,12.936622,25.424940
1,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,98.262171,111.370181,0.373016,0.611790,17.094738,16.444351
2,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.561335,88.314998,0.408802,0.676006,14.997233,17.232916
3,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,105.399202,74.307335,0.302568,0.640555,13.720675,22.729712
4,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,111.062402,110.135102,0.187558,0.462170,11.299311,16.773233
...,...,...,...,...,...,...,...,...,...
734,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.406627,84.377056,0.400957,0.676917,16.661667,23.462009
735,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,94.142142,102.236035,0.195535,0.459682,20.104237,20.941439
736,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,108.593665,73.421621,0.391609,0.822925,12.342820,25.101833
737,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,drive/MyDrive/ImageSuperResolution/DatasetBatc...,99.530373,89.219008,0.306131,0.529648,14.222706,19.163500


In [28]:
eva.to_csv('drive/MyDrive/ImageSuperResolution/evaluation_test.csv')

In [30]:
a = np.mean(eva['SRGAN_mse'])
b = np.mean(eva['FINAL_mse'])
print('Average Mean Squarred Error for SRGAN generated images is', a)
print('Average Mean Squarred Error for Final images is', b)

Average Mean Squarred Error for SRGAN generated images is 104.83373228743793
Average Mean Squarred Error for Final images is 89.74741085926924


In [29]:
a = np.mean(eva['SRGAN_ssim'])
b = np.mean(eva['FINAL_ssim'])

print('Average SSIM for SRGAN generated images is', a)
print('Average SSIM for Final images is', b)

Average SSIM for SRGAN generated images is 0.3327632575360138
Average SSIM for Final images is 0.6151859456010375


In [31]:
a = np.mean(eva['SRGAN_psnr'])
b = np.mean(eva['FINAL_psnr'])

print('Average PSNR for SRGAN generated images is', a)
print('Average PSNR for Final images is', b)

Average PSNR for SRGAN generated images is 14.178435739870807
Average PSNR for Final images is 20.524313993802412
